In [1]:
import pandas as pd
import numpy as np

In [ ]:
#Load the dataset
df=pd.read_csv("")

In [ ]:
df.head()

Check the overall details about the dataset

In [ ]:
df.info()

Check whether there are missing values in the dataset

In [ ]:
#Check missing values
df.isnull().sum()

Check whether there are duplicates in the dataset

In [ ]:
#Check for duplicate rows
df.duplicated().sum()